In [1]:
import numpy as np 
import pandas as pd 

import os
print(os.listdir("/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge"))

['sample_submission_zero.csv', 'transactions.csv.7z', 'user_logs_v2.csv', 'user_logs.csv', '.DS_Store', 'train.csv.7z', 'user_logs_v2.csv.7z', 'sample_submission_v2.csv', 'Untitled6.ipynb', 'WSDMChurnLabeller.scala', 'members_v3.csv.7z', 'train_v2.csv', 'data 4', 'data 3', 'user_logs.csv.7z', 'data 2', 'transactions_v2.csv', 'train.csv', '.ipynb_checkpoints', 'members_v3.csv', 'transactions.csv', 'data']


In [2]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import OneClassSVM
import collections

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

In [3]:
train = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/train.csv')
train = pd.concat((train, pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/train_v2.csv')),axis=0, ignore_index=True).reset_index(drop=True)
test = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/sample_submission_v2.csv')



In [4]:
transactions = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/transactions.csv', usecols=['msno'])
transactions = pd.concat((transactions, pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/transactions_v2.csv', usecols=['msno'])), axis=0, ignore_index=True).reset_index(drop=True)
transactions = pd.DataFrame(transactions['msno'].value_counts().reset_index())
transactions.columns = ['msno','trans_count']
train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')

In [5]:
transactions = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/transactions_v2.csv') 
transactions = transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
transactions = transactions.drop_duplicates(subset=['msno'], keep='first')

train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions=[]



In [6]:
user_logs = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/user_logs_v2.csv', usecols=['msno'])
user_logs = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
user_logs.columns = ['msno','logs_count']
train = pd.merge(train, user_logs, how='left', on='msno')
test = pd.merge(test, user_logs, how='left', on='msno')

user_logs = []; 

In [7]:
def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

In [8]:
last_user_logs = []
last_user_logs.append(transform_df(pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/user_logs_v2.csv')))
last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = transform_df2(last_user_logs)
print ('merging user logs features...')
train = pd.merge(train, last_user_logs, how='left', on='msno')
test = pd.merge(test, last_user_logs, how='left', on='msno')
last_user_logs=[]

merging user logs features...


In [9]:
members = pd.read_csv('/Users/bainaonao/Desktop/kkbox-churn-prediction-challenge/members_v3.csv')
train = pd.merge(train, members, how='left', on='msno')
test = pd.merge(test, members, how='left', on='msno')
members = []; print('members merge...') 

members merge...


In [10]:
gender = {'male':1, 'female':2}
train['gender'] = train['gender'].map(gender)
test['gender'] = test['gender'].map(gender)

train = train.fillna(0)
test = test.fillna(0)

In [11]:
train = train.fillna(0)
test = test.fillna(0)
train.head()

,msno,is_churn,trans_count,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,...,num_75,num_985,num_100,num_unq,total_secs,city,bd,gender,registered_via,registration_init_time
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,18.0,36.0,2.0,9.0,20050406.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,281.600,10.0,38.0,1.0,9.0,20050407.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,11.0,27.0,2.0,9.0,20051016.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,3,40.0,30.0,149.0,149.0,1.0,20170327.0,20170426.0,...,5.0,4.0,19.0,51.0,6171.145,13.0,23.0,2.0,9.0,20051102.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,9,38.0,90.0,477.0,477.0,0.0,20170225.0,20170528.0,...,0.0,2.0,11.0,12.0,3132.042,3.0,27.0,1.0,9.0,20051228.0


In [12]:
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LambdaCallback
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Lambda
from keras.layers.core import Dropout
from keras import regularizers
from keras.models import Model, load_model
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling1D


In [13]:
cols = [c for c in train.columns if c not in ['is_churn','msno']]
X_train = StandardScaler().fit_transform(train[cols].values)
y_train = train['is_churn'].values
X_test = StandardScaler().fit_transform(test[cols].values)

In [14]:
lsize = 128
model = Sequential()
model.add(Dense(lsize, input_dim=int(X_train.shape[1]),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(Dense(int(lsize/2), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.25))
model.add(Dense(int(lsize/4),kernel_regularizer=regularizers.l2(0.1), activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               3072      
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [15]:
history = model.fit(X_train, y_train, epochs=10, batch_size=1026,#512, 
                    validation_split=0.2, verbose=1)

Epoch 1/10


2023-04-18 14:47:07.438029: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1532/1532 [==============================] - 11s 7ms/step - loss: 4.8450 - accuracy: 0.6905 - val_loss: 4.6448 - val_accuracy: 0.8673
Epoch 2/10
1532/1532 [==============================] - 8s 5ms/step - loss: 4.5260 - accuracy: 0.7984 - val_loss: 4.3036 - val_accuracy: 0.9261
Epoch 3/10
1532/1532 [==============================] - 8s 5ms/step - loss: 4.1751 - accuracy: 0.8723 - val_loss: 3.9497 - val_accuracy: 0.9388
Epoch 4/10
1532/1532 [==============================] - 8s 5ms/step - loss: 3.8178 - accuracy: 0.9117 - val_loss: 3.6005 - val_accuracy: 0.9430
Epoch 5/10
1532/1532 [==============================] - 8s 5ms/step - loss: 3.4646 - accuracy: 0.9315 - val_loss: 3.2570 - val_accuracy: 0.9437
Epoch 6/10
1532/1532 [==============================] - 8s 5ms/step - loss: 3.1232 - accuracy: 0.9412 - val_loss: 2.9302 - val_accuracy: 0.9442
Epoch 7/10
1532/1532 [==============================] - 8s 5ms/step - loss: 2.7979 - accuracy: 0.9466 - val_loss: 2.6209 - val_accuracy: 0.9442
Ep

In [16]:
predictions = model.predict(X_test)
test['is_churn'] = predictions.clip(0.+1e-15, 1-1e-15)
test[['msno','is_churn']].to_csv('submission.csv', index=False)

28359/28359 [==============================] - 10s 365us/step
